# Deep Learning: Assignment 2

In [1]:
# Define imports & defaults
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

# import helper functions
import os
import sys

sys.path.append(os.getcwd())
from a02helper import *

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)  # ensure reproducibility
np.random.seed(0)

MAX_SEQ_LEN = 200
BATCH_SIZE = 32

## Task 1: Datasets

In [2]:
import string
from torchtext.vocab import vocab
from torchtext.data import get_tokenizer

class ReviewsDataset(Dataset):
    def __init__(
        self,
        reviews_file="data/reviews_small.txt",
        labels_file="data/labels_small.txt",
        use_vocab=False,
    ):
        """
        A dataset of movie reviews and their labels.

        Args:
            reviews_file: the reviews file
            labels_file:  the labels file
            use_vocab: if True, yield reviews in a numerical representation
        """
        # Load data from filesystem
        with open(reviews_file) as f:
            raw_reviews = f.readlines()

        with open(labels_file) as f:
            raw_labels = f.readlines()

        # Preprocessing and store (in memory)
        self._reviews = self._preprocess_reviews(raw_reviews)
        self._labels = self._preprocess_labels(raw_labels)

        # Build vocabulary
        self.vocab = None
        if use_vocab:
            from torchtext.vocab import build_vocab_from_iterator
            self.vocab = build_vocab_from_iterator(
                self._reviews, specials=["<pad>"]  # will get token id 0
            )

    def __len__(self):
        """Returns the length of the dataset."""
        return len(self._reviews)

    def __getitem__(self, idx):
        """
        Returns a tuple of a preprocessed review and the corresponding label. If the
        vocabulary is enabled, returns a numerical representation of the review.

        Args:
            idx: a single index
        Returns: a (review, label) tuple
        """
        review = self._reviews[idx]
        label = self._labels[idx]        
        if self.vocab:
            review_ids = [self.vocab.get_stoi()[token] for token in review]
            return review_ids, label
        else:
            return review, label

    def _preprocess_reviews(self, raw_reviews):
        """
        Applies two kinds of preprocessing:

        (i) Apply the "basic_english" tokenizer from the torchtext library to
        transform every review into a list of normalized tokens (cf.
        https://pytorch.org/text/stable/data_utils.html#get-tokenizer).

        (ii) Remove punctuation (cf.
        https://docs.python.org/3/library/string.html#string.punctuation).

        Returns: list of tokenized reviews
        """
        tokenizer = get_tokenizer("basic_english")
        tokenized_list = []
        for review in range (len(raw_reviews)):
            temp_str = raw_reviews[review].translate(str.maketrans("", "", string.punctuation))
            temp_str = tokenizer(temp_str)
            tokenized_list.append(temp_str)
        return tokenized_list

    def _preprocess_labels(self, raw_labels):
        """
        Transform raw labels into integers, where 1="positive" and 0 otherwise.

        Returns: list of labels
        """
        # Hint: You can remove leading and trailing whitespace from the raw labels using
        # the strip() method.
        preprocessed_labels_list = []
        for label in raw_labels:
            label = label.strip()
            if label == "positive":
                preprocessed_labels_list.append(1)
            else:
                preprocessed_labels_list.append(0)
        return preprocessed_labels_list

In [3]:
# Test your code (without vocabulary).
dataset = ReviewsDataset()
print(dataset[0])

# Should yield:
# (['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', ... ], 1)

(['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', 'such', 'as', 'teachers', 'my', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', 'high', 's', 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', 'teachers', 'the', 'scramble', 'to', 'survive', 'financially', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', 'pomp', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', 'i', 'immediately', 'recalled', 'at', 'high', 'a', 'classic', 'line', 'inspector', 'i', 'm', 'here', 'to', 'sack', 'one', 'of', 'your', 'teachers', 'student', 'welcome', 'to', 'brom

In [4]:
# Test your code (with vocabulary).
dataset = ReviewsDataset(use_vocab=True)
print(dataset[0])

# Should yield:
# ([10661, 307, 6, 3, 1177, 202, 8,  ... ], 1)

([10661, 307, 6, 3, 1177, 202, 8, 2248, 33, 1, 168, 56, 15, 49, 85, 8902, 43, 422, 122, 140, 15, 3234, 59, 144, 9, 1, 5504, 6267, 454, 72, 5, 260, 12, 10661, 307, 13, 2060, 6, 73, 2780, 5, 692, 76, 6, 3234, 1, 29527, 5, 1730, 7117, 1, 6161, 1726, 36, 52, 68, 212, 143, 63, 1409, 3234, 17974, 1, 28056, 4, 1, 221, 758, 31, 2748, 72, 4, 1, 6311, 10, 731, 2, 63, 1726, 54, 10, 208, 1, 321, 9, 64, 3, 1601, 4042, 743, 5, 2853, 187, 1, 422, 10, 1254, 10116, 33, 307, 3, 380, 322, 6162, 10, 135, 136, 5, 10172, 30, 4, 134, 3234, 1601, 2545, 5, 10661, 307, 10, 529, 12, 113, 1841, 4, 59, 676, 103, 12, 10661, 307, 6, 227, 4163, 48, 3, 2201, 12, 8, 231, 21], 1)


## Task 2: Data Loaders

In [5]:
# Split dataset into training, validation, and test subsets
dataset = ReviewsDataset(use_vocab=True)
train_set, val_set, test_set = torch.utils.data.random_split(
    dataset, [0.8, 0.1, 0.1], generator=torch.Generator().manual_seed(123)
)

### Task 2a

In [6]:
# Example usage of a data loader
dataloader = DataLoader(
    val_set,  # a dataset
    1,  # desired batch size
    False,  # whether to randomly shuffle the dataset
    num_workers=0,  # number of workers that construct batches in parallel
)

In [7]:
# Let's print the first batch
batch = next(iter(dataloader))
print(batch)
# [[tensor([11]), tensor([6]), tensor([1]), ...], tensor([0])]

[[tensor([11]), tensor([6]), tensor([1]), tensor([1037]), tensor([6578]), tensor([4]), tensor([10]), tensor([89]), tensor([120]), tensor([48]), tensor([163]), tensor([47]), tensor([6]), tensor([27]), tensor([342]), tensor([4]), tensor([2228]), tensor([140]), tensor([3]), tensor([186]), tensor([1466]), tensor([1]), tensor([771]), tensor([26]), tensor([78]), tensor([1459]), tensor([200]), tensor([1101]), tensor([1]), tensor([66]), tensor([26]), tensor([78]), tensor([5199]), tensor([5]), tensor([2288]), tensor([1]), tensor([7861]), tensor([6591]), tensor([2]), tensor([83]), tensor([2446]), tensor([25]), tensor([1]), tensor([182]), tensor([2756]), tensor([2520]), tensor([34]), tensor([1]), tensor([145]), tensor([8]), tensor([13]), tensor([39]), tensor([290]), tensor([25]), tensor([252]), tensor([14480]), tensor([32]), tensor([52]), tensor([497]), tensor([9]), tensor([223]), tensor([1]), tensor([3254]), tensor([25]), tensor([937]), tensor([2]), tensor([153]), tensor([568]), tensor([5]), ten

In [8]:
print(len(val_set[0][0]))
print(len(val_set[1][0]))
print(len(val_set[2][0]))

116
158
131


Having different length of the input reviews, during the batch size, it will require those input sentence are having the same length to represent their representation

### Task 2b

In [9]:
def review_collate_fn(raw_batch):
    """Prepare batches of reviews from a review dataset.

    Args:
        raw_batch: collection of (review, label)-tuples from a ReviewDataset

    Returns: a tuple (review x token id tensor, label tensor) of sizes
    batch_size*MAX_SEQ_LEN and batch_size, respectively.

    """
    reviews, labels = zip(*raw_batch)
    padded_reviews = torch.zeros(len(reviews), MAX_SEQ_LEN, dtype=torch.long)
    
    for i, review in enumerate (reviews):
        if len(review) <= MAX_SEQ_LEN:
            padded_reviews[i, :len(review)] = torch.tensor(review)
        else:
            padded_reviews[i, :] = torch.tensor(review[:MAX_SEQ_LEN]).clone().detach()
    label_tensor = torch.tensor(labels, dtype=torch.long)
    return padded_reviews, label_tensor

In [10]:
# Test your function
review_collate_fn([([1, 2, 3], 1), (torch.arange(MAX_SEQ_LEN * 2) + 1, 0)])

# Should yield:
# (tensor([[  1,   2,   3,   0,   0,  ..., 0 ],
#          [  1,   2,   3,   4,   5, ..., 200 ]]),
#  tensor([1, 0]))

/var/folders/2h/v65x911156j8d_r4wnykkqrw0000gn/T/ipykernel_40995/3549971076.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_reviews[i, :] = torch.tensor(review[:MAX_SEQ_LEN]).clone().detach()


(tensor([[  1,   2,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0, 

In [11]:
# Create the data loaders (with shuffling for training data -> randomization)
train_loader = DataLoader(train_set, BATCH_SIZE, True, collate_fn=review_collate_fn)
val_loader = DataLoader(val_set, BATCH_SIZE, False, collate_fn=review_collate_fn)
test_loader = DataLoader(test_set, BATCH_SIZE, False, collate_fn=review_collate_fn)

In [12]:
# Let's print the first batch
batch = next(iter(val_loader))
print(batch)


# (tensor([[   11,     6,     1,  ...,     0,     0,     0],
#         [   11,   170,  2220,  ...,     0,     0,     0],
#         [   48,     3, 30376,  ...,     0,     0,     0],
#         ...,
#         [  176,    56,    10,  ...,     0,     0,     0],
#         [  239,   534,  1404,  ...,    44,   120,     1],
#         [ 2954, 15576,     6,  ...,  2678,    65,     1]]),
#  tensor([0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1,
#         0, 0, 0, 1, 1, 1, 0, 0]))

(tensor([[   11,     6,     1,  ...,     0,     0,     0],
        [   11,   170,  2220,  ...,     0,     0,     0],
        [   48,     3, 30376,  ...,     0,     0,     0],
        ...,
        [  176,    56,    10,  ...,     0,     0,     0],
        [  239,   534,  1404,  ...,    44,   120,     1],
        [ 2954, 15576,     6,  ...,  2678,    65,     1]]), tensor([0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 1, 1, 1, 0, 0]))


## Task 3: Recurrent Neural Networks

In [13]:
class SimpleLSTM(nn.Module):
    def __init__(
        self, vocab_size, embedding_dim, hidden_dim, num_layers=1, cell_dropout=0.0
    ):
        """
        Initializes the model by setting up the layers.

        Args:
            vocab_size: number of unique words in the reviews
            embedding_dim: size of the embeddings
            hidden_dim: dimension of the LSTM output
            num_layers: number of LSTM layers
            cell_dropout: dropout applied between the LSTM layers
                          (provide to LSTM constructor as dropout argument)
        """
        super(SimpleLSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first = True, num_layers=self.num_layers, dropout=cell_dropout)
        self.cell_dropout = cell_dropout
        self.fc = nn.Linear(self.hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()
        

    def forward(self, x):
        """
        Performs a forward pass of the model on some input and hidden state.

        Parameters
        ----------
        x: batch as a (batch_size, sequence_length) tensor

        Returns
        -------
        Probability of positive class and the last output of the LSTM.
        """
        hidden = self.init_hidden(len(x))
        embeddings = self.word_embeddings(x)
        outputs, (h_n, c_n) = self.lstm(embeddings, hidden)
        last_output = h_n[-1, :, :]
        logits = self.fc(last_output)
        predictions = self.sigmoid(logits)
        return predictions, last_output

    def init_hidden(self, batch_size):
        """Initialize hidden states.

        Returns a tuple of two num_layers x batch_size x hidden_dim tensors (one for
        initial cell states, one for initial hidden states) consisting of all zeros.
        """
        # Note: ensure that the returned tensors are located on DEVICE.
        device = self.word_embeddings.weight.device
        h_0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=device)
        c_0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=device)
        return h_0, c_0


# Test constructor
model = SimpleLSTM(50, 10, 32, 2, 0.1).to(DEVICE)
print(model)

# Should give:
# SimpleLSTM(
#   (embedding): Embedding(50, 10)
#   (lstm): LSTM(10, 32, num_layers=2, batch_first=True, dropout=0.1)
#   (fc): Linear(in_features=32, out_features=1, bias=True)
#   (sigmoid): Sigmoid()
# )

SimpleLSTM(
  (word_embeddings): Embedding(50, 10)
  (lstm): LSTM(10, 32, num_layers=2, batch_first=True, dropout=0.1)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [14]:
# Test forward pass
model = SimpleLSTM(50, 10, 32, 2).to(DEVICE)
dummy_data = torch.arange(30, dtype=torch.int, device=DEVICE).reshape(3, 10)

# fix model parameters
for key in model.state_dict():
    model.state_dict()[key][:] = 0.1
probs, states = model(dummy_data)
print(probs)
print(states)


# tensor([[0.9643],
#         [0.9643],
#         [0.9643]], device='cuda:0 or cpu', grad_fn=<SigmoidBackward0>)
# tensor([[0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985,
#          0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985,
#          0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985,
#          0.9985, 0.9985, 0.9985, 0.9985, 0.9985],
#         [0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985,
#          0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985,
#          0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985,
#          0.9985, 0.9985, 0.9985, 0.9985, 0.9985],
#         [0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985,
#          0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985,
#          0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985,
#          0.9985, 0.9985, 0.9985, 0.9985, 0.9985]], device='cuda:0 or cpu',
#        grad_fn=<SliceBackward0>)

tensor([[0.9643],
        [0.9643],
        [0.9643]], grad_fn=<SigmoidBackward0>)
tensor([[0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985,
         0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985,
         0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985,
         0.9985, 0.9985, 0.9985, 0.9985, 0.9985],
        [0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985,
         0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985,
         0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985,
         0.9985, 0.9985, 0.9985, 0.9985, 0.9985],
        [0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985,
         0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985,
         0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985, 0.9985,
         0.9985, 0.9985, 0.9985, 0.9985, 0.9985]], grad_fn=<SliceBackward0>)


### Task 3d

In [15]:
@torch.no_grad()  # Disables autograd for this function
def reviews_eval(
    model, eval_loader, label="val", loss_fn=torch.nn.functional.binary_cross_entropy
):
    model.to(DEVICE)
    model.eval()  # sets model to evaluation mode (e.g., relevant for dropout)

    total_correct = total_loss = 0
    for reviews, labels in eval_loader:
        reviews, labels = reviews.to(DEVICE), labels.to(DEVICE)

        # Forward pass: Compute the model's output, reshape it to a vector, and
        # then run the provided loss function.
        prediction, _ = model(reviews)
        prediction = prediction.view(-1)
        loss = loss_fn(prediction, labels.float())
        
        # Eval stats: Add loss to total_loss and number of correct predictions to
        # total_correct.
        total_loss = total_loss + loss.item()
        prediction_class = torch.round(prediction)
        correct = (prediction_class == labels).sum().item()
        total_correct = total_correct + correct

    print(
        f"            "
        f"{label} accuracy: {total_correct / len(eval_loader.dataset):.4f}\t"
        f"{label} loss: {total_loss / len(eval_loader):.4f}"
    )

In [ ]:
# Test your implementation
model = SimpleLSTM(len(dataset.vocab), 10, 10, 1, 0).to(DEVICE)
reviews_eval(model, val_loader)

# Should yield with different but similar numbers:
#             val accuracy: 0.5100	val loss: 0.6928

### Task 3e

In [ ]:
def reviews_train(
    model,
    train_loader,
    val_loader,
    lr=0.01,
    epochs=3,
    max_norm=5,
    loss_fn=torch.nn.functional.binary_cross_entropy,
):
    """
    Train a network on the review data

    Args:
        model: Initialized model
        train_loader: Dataloader for the training data
        val_loader: Dataloader for the validation data
        lr: learning rate
        epochs: number of epochs
        max_norm: max norm of gradients for gradient clipping
        loss_fn: Loss function
    """
    # Send the model's parameters to your accelerator (cuda or cpu)
    model = model.to(DEVICE)

    # Define optimizer for the parameters which require gradients (cf. Task 5)
    optimizer = torch.optim.Adam(
        [param for param in model.parameters() if param.requires_grad], lr=lr
    )

    # Let's go
    for epoch in range(epochs):
        total_correct = total_loss = 0
        for reviews, labels in train_loader:
            model.train()

            # Send batch to your accelerator
            reviews, labels = reviews.to(DEVICE), labels.to(DEVICE)

            # Forward pass: Compute the model's output, reshape it to a vector, and then
            # run the provided loss function.
            prediction, _ = model(reviews)
            prediction = prediction.view(-1)
            loss = loss_fn(prediction, labels.float())

            # Backward pass:
            # (i)   Compute the gradients wrt. the loss
            # (ii)  Clip the gradients using
            #       https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html to max_norm
            # (iii) Run the optimizer
            # (iv)  Clear all accumulated gradients
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = max_norm, norm_type = 2.0)
            optimizer.step()
            optimizer.zero_grad()


            # Compute epoch statistics:
            # (i)  Add the loss of this batch to the total_loss
            # (ii) Add the number of correct predictions (max prob) to total_correct
            total_loss = total_loss + loss.item()
            prediction_class = torch.round(prediction)
            correct = (prediction_class == labels).sum().item()
            total_correct = total_correct + correct

        print(
            f"Epoch {epoch + 1:2}\t"
            f"train accuracy: {total_correct / len(train_loader.dataset):.4f}\t"
            f"train loss: {total_loss / len(train_loader):.4f}"
        )

        # now validate
        reviews_eval(model, val_loader, loss_fn=loss_fn)

In [20]:
# Test your implementation
model = SimpleLSTM(len(dataset.vocab), 10, 10, 1).to(DEVICE)
reviews_train(model, train_loader, val_loader, epochs=5)

# Should yield something like (note: numbers have high variance over runs):
# Epoch  1	train accuracy: 0.4994	train loss: 0.6953
#             val accuracy: 0.4875	val loss: 0.6922
# Epoch  2	train accuracy: 0.5319	train loss: 0.6885
#             val accuracy: 0.5275	val loss: 0.6861
# Epoch  3	train accuracy: 0.6059	train loss: 0.6443
#             val accuracy: 0.5400	val loss: 0.6902
# Epoch  4	train accuracy: 0.6863	train loss: 0.5438
#             val accuracy: 0.5925	val loss: 0.7453
# Epoch  5	train accuracy: 0.8334	train loss: 0.3875
#             val accuracy: 0.7300	val loss: 0.6310

KeyboardInterrupt: 

## Task 4: Pre-trained Embeddings & Visualization

### Task 4b

In [ ]:
# Load Glove embeddings into a plain embedding layer.
vocab = dataset.vocab
glove_embeddings = nn.Embedding(len(vocab), 100, device=DEVICE)
reviews_load_embeddings(glove_embeddings, vocab.get_stoi())

In [ ]:
# Print one embedding
glove_embeddings(torch.tensor(vocab["movie"], device=DEVICE))

In [ ]:
# Plot embeddings of first 100 words using t-SNE
nextplot()
_ = tsne_vocab(glove_embeddings, torch.arange(100), vocab)

In [ ]:
# You can also specify colors and/or drop the item labels
nextplot()
_ = tsne_vocab(glove_embeddings, torch.arange(100), colors=[0] * 50 + [1] * 50)

In [ ]:
# Note: you can obtain the embeddings tensor using glove_embeddings.weight.data
embeddings_tensor = glove_embeddings.weight.data

def plot_tsne(embeddings, words, colors=None):
    tsne = TSNE(n_components=2, perplexity=30, n_iter=1000, metric='cosine')
    embeddings_2d = tsne.fit_transform(embeddings)

    plt.figure()
    if colors:
        plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=colors)
    else:
        plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1])

    for i, word in enumerate(words):
        plt.annotate(word, (embeddings_2d[i, 0], embeddings_2d[i, 1]))

    plt.xlabel("t-SNE Dimension 1")
    plt.ylabel("t-SNE Dimension 2")
    plt.title("t-SNE Visualization of GloVe Embeddings")
    plt.show()

subset_words = vocab.get_itos()
subset_words = [subset_words[i] for i in range(100)]

plot_tsne(embeddings_tensor[:100], subset_words)

In [ ]:
import umap.umap_ as umap

def plot_umap(embeddings, words, colors=None):
  """
  Visualizes word embeddings using UMAP for dimensionality reduction.
  """
  mapper = umap.UMAP(n_components=2)  # Reduce to 2D
  embeddings_2d = mapper.fit_transform(embeddings)

  plt.figure()
  if colors:
      plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=colors)
  else:
      plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1])

  for i, word in enumerate(words):
      plt.annotate(word, (embeddings_2d[i, 0], embeddings_2d[:, 1]))

  plt.xlabel("UMAP Dimension 1")
  plt.ylabel("UMAP Dimension 2")
  plt.title("UMAP Visualization of GloVe Embeddings")
  plt.show()

# Example usage with UMAP
subset_words = vocab.get_itos()
subset_words = subset_words[:100]  # Select first 100 words

plot_umap(embeddings_tensor[:100], subset_words)

### Task 4c

In [ ]:
# hyperparameter settings for rest of task 4
vocab_size = len(dataset.vocab)
embedding_dim = 100
hidden_dim = 100
num_layers = 2
n_epochs = 10
cell_dropout = 0.0

In [ ]:
# train a plain model
model = SimpleLSTM(vocab_size, embedding_dim, hidden_dim, num_layers, cell_dropout).to(
    DEVICE
)
reviews_train(model, train_loader, val_loader, epochs=n_epochs)

# Should reach a (train) accuracy of >0.9. If not, rerun.

In [ ]:
# Plot t-SNE embeddings of the thought vectors for training data
# point color = label
nextplot()
_ = tsne_thought(model, train_loader, DEVICE)

In [ ]:
# Plot t-SNE embeddings of of the thought vectors for validation data
nextplot()
_ = tsne_thought(model, val_loader, DEVICE)

### Task 4d

In [ ]:
# Initialize the model with *p*re-trained embeddings with *f*inetuning, then train
model_pf = SimpleLSTM(vocab_size, embedding_dim, hidden_dim, num_layers, cell_dropout)
reviews_load_embeddings(model_pf.embedding, vocab.get_stoi())
reviews_train(model_pf, train_loader, val_loader, epochs=n_epochs)

### Task 4e

In [ ]:
# Initialize the model with *p*re-trained embeddings without finetuning, then train
model_p = SimpleLSTM(vocab_size, embedding_dim, hidden_dim, num_layers, cell_dropout)
reviews_load_embeddings(model_p.embedding, vocab.get_stoi())
model_p.embedding.weight.requires_grad = False
reviews_train(model_p, train_loader, val_loader, epochs=n_epochs)

## Task 5: Exploration

In [ ]:
# YOUR CODE HERE